# Scraping mobile phone reviews from amazon.in

In [43]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [44]:
search_query="smartphones"

In [45]:
base_url="https://www.amazon.in/s?k="

In [46]:
url=base_url+search_query

In [47]:
url

'https://www.amazon.in/s?k=smartphones'

In [48]:
header={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87 Safari/537.36'}

In [49]:
search_response=requests.get(url,headers=header)


In [50]:
search_response.status_code
#search_response.text

503

#### A function to get the content of the page of required query

In [51]:
cookie={} # insert request cookies within{}
def getAmazonSearch(search_query,page):
    url=base_url+search_query+'&pageNumber='+str(page)
    print(url)
    page=requests.get(url,cookies=cookie,headers=header)
    if page.status_code==200:
        return page
    else:
        return "Error"

#### A function to get the contents of individual product pages using 'data-asin' number (unique identification number)

In [52]:
def Searchasin(asin):
    url="https://www.amazon.in/dp/"+asin
    print(url)
    page=requests.get(url,cookies=cookie,headers=header)
    if page.status_code==200:
        return page
    else:
        return "Error"

#### A function to pass on the link of 'see all reviews' and extract the content

In [53]:
def Searchreviews(review_link):
    url="https://www.amazon.in"+review_link
    print(url)
    page=requests.get(url,cookies=cookie,headers=header)
    if page.status_code==200:
        return page
    else:
        return "Error"

In [59]:
import requests

def getAmazonSearch(keyword, page):
    url = f'https://www.amazon.in/s?k={keyword}&pageNumber={page}'
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    response = requests.get(url, headers=headers)
    return response

# Example of usage
response = getAmazonSearch('smartphones', 1)
print(type(response))
print(response.status_code)  # Now this should work
print(response.content)

import time

max_retries = 3
retry_delay = 5  # seconds

for retry in range(max_retries):
    try:
        response = getAmazonSearch('smartphones', 1)
        response.raise_for_status()  # Check if the request was successful
        break  # Break out of the loop if successful
    except requests.exceptions.HTTPError as e:
        if response.status_code == 503:
            print(f"Service Unavailable (503) - Retrying in {retry_delay} seconds...")
            time.sleep(retry_delay)
        else:
            print(f"HTTP Error: {e}")
            break  # Break out of the loop for other HTTP errors


<class 'requests.models.Response'>
503
b'<html>\n<head>\n<title>503 - Service Unavailable Error</title>\n</head>\n<body bgcolor="#FFFFFF" text="#000000">\n\n<!--\n        To discuss automated access to Amazon data please contact api-services-support@amazon.com.\n        For information about migrating to our APIs refer to our Marketplace APIs at https://developer.amazonservices.in/ref=rm_5_sv, or our Product Advertising API at https://affiliate-program.amazon.in/gp/advertising/api/detail/main.html/ref=rm_5_ac for advertising use cases.\n-->\n\n<center>\n<a href="https://www.amazon.in/ref=cs_503_logo/">\n<img src="https://images-eu.ssl-images-amazon.com/images/G/31/x-locale/communities/people/logo.gif" width=200 height=45 alt="Amazon.in" border=0></a>\n<p align=center>\n<font face="Verdana,Arial,Helvetica">\n<font size="+2" color="#CC6600"><b>Oops!</b></font><br>\n<b>It\'s rush hour and traffic is piling up on that page. Please try again in a short while.<br>If you were trying to place 

In [56]:
print(type(response))
print(response.status_code)  # Print the HTTP status code
print(response.content)      # Print the content of the response


<class 'str'>


AttributeError: 'str' object has no attribute 'status_code'

### First page product reviews extraction

In [54]:
product_names=[]
response=getAmazonSearch('smartphones',1)
soup=BeautifulSoup(response.content)

for i in soup.findAll("span",{'class':'a-size-medium a-color-base a-text-normal'}): # the tag which is common for all the names of products
    product_names.append(i.text) #adding the product names to the list

https://www.amazon.in/s?k=smartphones&pageNumber=1


AttributeError: 'str' object has no attribute 'content'

findall() is used to find all the tags with the mentioned attributes and values in a page

In [ ]:
product_names

["Samsung Galaxy M14 5G (ICY Silver,4GB,128GB)|50MP Triple Cam|Segment's Only 6000 mAh 5G SP|5nm Processor|2 Gen. OS Upgrade & 4 Year Security Update|12GB RAM with RAM Plus|Android 13|Without Charger",
 'Apple iPhone 13 (128GB) - Starlight',
 'realme narzo N53 (Feather Black, 8GB+128GB) 33W Segment Fastest Charging | Slimmest Phone in Segment | 90 Hz Smooth Display',
 'Xiaomi Redmi 10 Power (Power Black, 8GB RAM, 128GB Storage)',
 'Samsung Galaxy M13 (Midnight Blue, 4GB, 64GB Storage) | 6000mAh Battery | Upto 8GB RAM with RAM Plus',
 'Redmi Note 11T 5G (Matte Black, 6GB RAM, 128GB ROM)| Dimensity 810 5G | 33W Pro Fast Charging | Charger Included | Additional Exchange Offers',
 'Samsung Galaxy M04 Dark Blue, 4GB RAM, 128GB Storage | Upto 8GB RAM with RAM Plus | MediaTek Helio P35 Octa-core Processor | 5000 mAh Battery | 13MP Dual Camera',
 'Redmi 12C (Mint Green, 6GB RAM, 128GB Storage) | High Performance Mediatek Helio G85 | Big 17cm(6.71) HD+ Display with 5000mAh(typ) Battery',
 'Lava

In [ ]:
len(product_names)

22

##### The method of extracting data-asin numbers are similar to that of product names. Only the tag details have to be changed in findall()

In [ ]:
data_asin=[]
pages=8
for i in range(pages):
    response=getAmazonSearch('smartphones',i)
    soup=BeautifulSoup(response.content)
    for i in soup.findAll("div",{'class':"sg-col-20-of-24 s-result-item s-asin sg-col-0-of-12 sg-col-28-of-32 sg-col-16-of-20 sg-col sg-col-32-of-36 sg-col-12-of-16 sg-col-24-of-28"}):
        data_asin.append(i['data-asin'])
    for i in soup.findAll("div",{'class':"sg-col-20-of-24 s-result-item s-asin sg-col-0-of-12 sg-col-28-of-32 sg-col-16-of-20 AdHolder sg-col sg-col-32-of-36 sg-col-12-of-16 sg-col-24-of-28"}):
        data_asin.append(i['data-asin'])

https://www.amazon.in/s?k=smartphones&pageNumber=0
https://www.amazon.in/s?k=smartphones&pageNumber=1
https://www.amazon.in/s?k=smartphones&pageNumber=2
https://www.amazon.in/s?k=smartphones&pageNumber=3
https://www.amazon.in/s?k=smartphones&pageNumber=4
https://www.amazon.in/s?k=smartphones&pageNumber=5
https://www.amazon.in/s?k=smartphones&pageNumber=6
https://www.amazon.in/s?k=smartphones&pageNumber=7


In [ ]:
data_asin[:5]

[]

In [ ]:
from bs4 import BeautifulSoup

data_asin = []
pages = 8


for i in range(pages):
    response = getAmazonSearch('smartphones', i)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Update class names based on the current Amazon HTML structure
    for item in soup.select('.s-asin'):
        if 'data-asin' in item.attrs:
            data_asin.append(item['data-asin'])

    for item in soup.select('.AdHolder'):  # Check for ads with different class
        if 'data-asin' in item.attrs:
            data_asin.append(item['data-asin'])

print("Output:")
for i in range(pages):
    print(f"https://www.amazon.in/s?k=smartphones&pageNumber={i}")

print("Data_asin:", data_asin[:5])


https://www.amazon.in/s?k=smartphones&pageNumber=0
https://www.amazon.in/s?k=smartphones&pageNumber=1
https://www.amazon.in/s?k=smartphones&pageNumber=2
https://www.amazon.in/s?k=smartphones&pageNumber=3
https://www.amazon.in/s?k=smartphones&pageNumber=4
https://www.amazon.in/s?k=smartphones&pageNumber=5
https://www.amazon.in/s?k=smartphones&pageNumber=6
https://www.amazon.in/s?k=smartphones&pageNumber=7
Output:
https://www.amazon.in/s?k=smartphones&pageNumber=0
https://www.amazon.in/s?k=smartphones&pageNumber=1
https://www.amazon.in/s?k=smartphones&pageNumber=2
https://www.amazon.in/s?k=smartphones&pageNumber=3
https://www.amazon.in/s?k=smartphones&pageNumber=4
https://www.amazon.in/s?k=smartphones&pageNumber=5
https://www.amazon.in/s?k=smartphones&pageNumber=6
https://www.amazon.in/s?k=smartphones&pageNumber=7
Data_asin: ['B0BZCSMP95', 'B0CHXDW1B6', 'B0CKN56PQ8', 'B09Y64H8VS', 'B0B4F52B5X']


In [ ]:
data_asin[:5]

['B0BZCSMP95', 'B0CHXDW1B6', 'B0CKN56PQ8', 'B09Y64H8VS', 'B0B4F52B5X']

In [ ]:
len(data_asin)


224

##### By passing the data-asin numbers, we can extract the 'see all reviews' link for each product in the page

In [ ]:
links = []
mobile_names = []
asins = []

for i in range(len(data_asin)):
    response = Searchasin(data_asin[i])
    soup = BeautifulSoup(response.content, 'html.parser')
    
    mobile = soup.find("span", {"id": "productTitle"})

    # Check if the element exists before accessing its content
    if mobile:
        mobile_names.append(mobile.get_text(strip=True))
    else:
        mobile_names.append("Title not found")

    link = soup.find("a", {'data-hook': "see-all-reviews-link-foot"})

    # Check if the element exists before accessing its attribute
    if link:
        links.append(link['href'])
    else:
        links.append("Link not found")

    asins.append(data_asin[i])

# Print the output
for i in range(len(asins)):
    print(f"ASIN: {asins[i]}, Name: {mobile_names[i]}, Link: {links[i]}")


https://www.amazon.in/dp/B0BZCSMP95
https://www.amazon.in/dp/B0CHXDW1B6
https://www.amazon.in/dp/B0CKN56PQ8
https://www.amazon.in/dp/B09Y64H8VS
https://www.amazon.in/dp/B0B4F52B5X
https://www.amazon.in/dp/B09LHYZ3GJ
https://www.amazon.in/dp/B0BMGG6NKT
https://www.amazon.in/dp/B0BYN6SGHD
https://www.amazon.in/dp/B0CHRPMSP1
https://www.amazon.in/dp/B0C45VWDWJ
https://www.amazon.in/dp/B09G9D8KRQ
https://www.amazon.in/dp/B0CHX1W1XY
https://www.amazon.in/dp/B0C46FN82F
https://www.amazon.in/dp/B07WHRBD95
https://www.amazon.in/dp/B0BBM7L888
https://www.amazon.in/dp/B0C2VH4DPM
https://www.amazon.in/dp/B0C74PD9VG
https://www.amazon.in/dp/B07WFPLX4V
https://www.amazon.in/dp/B07WGPKSG8
https://www.amazon.in/dp/B0BBM78DMV
https://www.amazon.in/dp/B0CHX7V9V9
https://www.amazon.in/dp/B0CHWZCY43
https://www.amazon.in/dp/B0BZCSMP95
https://www.amazon.in/dp/B0CHXDW1B6
https://www.amazon.in/dp/B09G9D8KRQ
https://www.amazon.in/dp/B0CHX1W1XY
https://www.amazon.in/dp/B0CHX7V9V9
https://www.amazon.in/dp/B0C

In [ ]:
print(len(links),len(mobile_names),len(asins))
mobile_names[80]



224 224 224


'Apple iPhone 15 Pro Max (1 TB) - Blue Titanium'

##### Now we have the 'see all review' links. Using this link along with a page number, we can extract the reviews in any number of pages for all the products

In [ ]:

reviews = []

for j in range(len(asins)):
    for k in range(10):
        response = Searchreviews(links[j]+'&pageNumber='+str(k))
        soup = BeautifulSoup(response.content, 'html.parser')

        titles = soup.findAll("a", {'data-hook': "review-title"})
        bodies = soup.findAll("span", {'data-hook': "review-body"})
        stars = soup.findAll("i", {'data-hook': "review-star-rating"})

        for i in range(len(titles)):
            reviews.append([mobile_names[j], asins[j], titles[i].text, bodies[i].text, int(stars[i].text[0])])

print('Number of reviews:', len(reviews))


https://www.amazon.in/Samsung-Segments-Smartphone-Octa-Core-Processor/product-reviews/B0BZCSMP95?reviewerType=all_reviews&pageNumber=0
https://www.amazon.in/Samsung-Segments-Smartphone-Octa-Core-Processor/product-reviews/B0BZCSMP95?reviewerType=all_reviews&pageNumber=1
https://www.amazon.in/Samsung-Segments-Smartphone-Octa-Core-Processor/product-reviews/B0BZCSMP95?reviewerType=all_reviews&pageNumber=2
https://www.amazon.in/Samsung-Segments-Smartphone-Octa-Core-Processor/product-reviews/B0BZCSMP95?reviewerType=all_reviews&pageNumber=3
https://www.amazon.in/Samsung-Segments-Smartphone-Octa-Core-Processor/product-reviews/B0BZCSMP95?reviewerType=all_reviews&pageNumber=4
https://www.amazon.in/Samsung-Segments-Smartphone-Octa-Core-Processor/product-reviews/B0BZCSMP95?reviewerType=all_reviews&pageNumber=5
https://www.amazon.in/Samsung-Segments-Smartphone-Octa-Core-Processor/product-reviews/B0BZCSMP95?reviewerType=all_reviews&pageNumber=6
https://www.amazon.in/Samsung-Segments-Smartphone-Octa-

ConnectionError: HTTPSConnectionPool(host='www.amazon.inlink%20not%20found&pagenumber=0', port=443): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x00000216666FB190>: Failed to resolve 'www.amazon.inlink%20not%20found&pagenumber=0' ([Errno 11001] getaddrinfo failed)"))

In [ ]:

review_data = pd.DataFrame(reviews, columns = ['mobile_names','asin','title', 'body','star'])  
print(review_data.shape)

(0, 5)


In [ ]:
review_data.sample(5)

ValueError: a must be greater than 0 unless no samples are taken

# Save to CSV

In [ ]:
review_data.to_csv('raw_mobile_reviews.csv',index=False)
